# Map preprocessing

In [2]:
import networkx as nx
import osmnx as ox

In [3]:
#Load map
g=ox.graph_from_place('Paris, France', network_type='bike',simplify=False)
gdf_nodes, gdf_edges=ox.graph_to_gdfs(g)

#Prune road types
remove_tags=['bridleway','path','unclassified','road']

del_list=[]
for j,i in gdf_edges.iterrows():
    if i["highway"] in remove_tags:
        del_list.append(j)
gdf_edges=gdf_edges.drop(index=del_list)
g=ox.utils_graph.graph_from_gdfs(gdf_nodes, gdf_edges, graph_attrs=None)  

#Rebuild the map
largest_cc = max(nx.strongly_connected_components(g), key=len)
g = g.subgraph(largest_cc)
gdf_nodes, gdf_edges=ox.graph_to_gdfs(g)

# Data generation

In [62]:
from DataGenerator import DataGenerator
import csv
import ast

In [76]:
def save_data(tracks,append=False):
    mode = "a" if append else "w"
    with open("data/generated-data.csv", mode) as f:  
        writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')
        for track in tracks:
            #Write item to outcsv
            writer.writerow(track)

def load_data():
    with open('data/generated-data.csv', 'r') as f:
        tracks=[]
        for track in f:
            entries = track.strip().split(';')
            nested_lists = [ast.literal_eval(entry) for entry in entries]
            tracks.append(nested_lists)
    return tracks

In [78]:
#Create the generator
path="data/comptage-velo-donnees-compteurs.csv"
datagen=DataGenerator(G=g,gdf_nodes_edges=(gdf_nodes,gdf_edges),data_path=path)

#Generate data in bacthes and save it to the generated data
generator_batch_size=10
n_batches=10
for batch in range(n_batches):
    tracks=datagen.track_generator(10,sampling_rate=1)
    save_data(tracks,append=True)

In [82]:
#Load the generated data
T=load_data()

# Travel time distribution